https://steemit.com/dclick/@anpigon/-textrank-summariser-1540351206980

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
df = pd.read_csv(f"{DATA_PATH}아모레크롤링_완료_필터링_수정.csv")
df

,상품분류,브랜드명,상품명,사용자 이름,별점,리뷰작성날짜,나이,성별,피부타입,피부트러블,리뷰
0,스킨케어,설화수,나의 첫 설화수,chob****,5,2023.08.13,30대,여성,복합성,민감성,"엄마가 좋아하는 에센스에요! 한방향나는 것도 좋고, 크림보단 에센스 좋은제품으로 바..."
1,스킨케어,설화수,나의 첫 설화수,moyr*,5,2023.08.13,50대 이상,여성,건성,탄력없음,더운 여름에는 윤조에센스 한 가지만 발라도 넘 촉촉합니다 이 나이에 피부 좋다는 소...
2,스킨케어,설화수,나의 첫 설화수,bobl*****,5,2023.08.13,50대 이상,여성,복합성,민감성,윤조에센스는 저의 가장 최애템이예요. 지금까지 수년간 지속적으로 빠짐없이 챙겨바르는...
3,스킨케어,설화수,나의 첫 설화수,mije******,5,2023.08.12,40대,여성,건성,주름,받아보니좋아요사진올리려했는데 안되서아쉽네요
4,스킨케어,설화수,나의 첫 설화수,hell****,5,2023.08.12,30대,여성,복합성,칙칙함,패키지 바뀌고 처음 사봤는데 좀더 세련되고 예쁜 느낌이에요 사계절 다 좋지만 여름에...
...,...,...,...,...,...,...,...,...,...,...,...
37813,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,juny*,5,2023.04.27,40대,여성,복합성,건조함,평소 아모레퍼시픽 바이탈뷰티의 콜라겐을 꾸준히 섭취하고 있는데 이번에 콜라겐 젤리가...
37814,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,spor***,5,2023.04.27,40대,여성,건성,칙칙함,신상품이라 구매해봤어요 먹기좋고 맛이 좋아 간식으로 먹으면 좋을거같아요 입심심할때...
37815,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,a010********,5,2023.04.21,40대,여성,건성,주름,애들이 더 좋아해서 재구매했어요 곤약젤리 느낌이랑 똑같다고 좋아하네요 애들이랑 함...
37816,뷰티푸드,바이탈뷰티,슈퍼콜라겐 구미 5입,Pny0***,5,2023.04.20,30대,여성,복합성,주름,먹기 편하고 맛있어요 간식먹고싶을때 먹는데 피부까지 좋아진다니 계속 시켜먹어야겠어요...


In [ ]:
! pip install keybert
! pip install krwordrank

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23767 sha256=89d99d63b2dcdb911b48a32786efdc6fe8f06df29e15d5006ec0ae95be7bfbeb
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=66c36789769f1a39fe26400913821d815026a042919e423f473c50b02d4d9182
 

# 요약

In [ ]:
! pip install konlpy
! pip install scikit-learn

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (465 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for textrank: filename=textrank-0.1.0-py3-none-any.whl size=3394 sha256=209b1392c231b3ab057491ef9cfc5273ac000677a98786ff904b311a018ae83b
  Stored in directory: /root/.cache/pip/wheels/d8/aa/86/ec56351537ee09443f2ec529eb340daa93bc761a21e362df69
Successfully built textrank


In [ ]:
! pip install jpype1
! pip install textrank

In [ ]:
from konlpy.tag import Okt
from collections import Counter
from textrank import KeywordSummarizer

# 사용자 입력으로부터 상품명을 받아옴
title = input("상품명을 입력하세요: ")

# 해당 상품명에 해당하는 리뷰들을 모아서 리스트로 가져옴
reviews = df[df['상품명'] == title]['리뷰'].to_list()

# 모든 리뷰 문장을 하나의 문자열로 합침
all_reviews = ' '.join(reviews)

# 키워드 추출
okt = Okt()
nouns = okt.nouns(all_reviews)
count = Counter(nouns)
keywords = [keyword for keyword, freq in count.most_common(5)]

# 키워드를 포함한 문장 요약
summarizer = KeywordSummarizer(tokenize = okt.morphs, min_count=2, verbose=True)
summarizer.fit(all_reviews)
summary = summarizer.summarize(keywords, topk=1)

print("상품명:", title)
print("추출된 키워드:", keywords)
print("요약 문장:")
print(summary)

ImportError: ignored

In [ ]:
!pip install kiwipiepy

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords =  Stopwords()

In [ ]:
twitter = Twitter()

def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

In [ ]:
nouns = get_nouns(sentences)

In [ ]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [ ]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)

In [ ]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

    words_graph, idx2word = build_words_graph(nouns)

In [ ]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [ ]:
sent_rank_idx = get_ranks(sent_graph)  #sent_graph : sentence 가중치 그래프
sent_rank_idx

In [ ]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)

In [ ]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

In [ ]:
def summarize(sent_num=1):
    summary = []
    index=[]
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)

    index.sort()
#     print(index)

    for idx in index:
        summary.append(sentences[idx])

    for text in summary :
        print(text)
        print("\n")

In [ ]:
summarize()

In [ ]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

    words_graph, idx2word = build_words_graph(nouns)

In [ ]:
keywords()

#요약 다시

In [ ]:
!pip install newspaper3k

In [ ]:
!pip install --upgrade pip

In [ ]:
!apt-get install python3-dev

In [ ]:
!pip install gensim==3.4.0

In [ ]:
from gensim.summarization.summarizer import summarize
from newspaper import Article

In [ ]:
!pip install bert-extractive-summarizer
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 사전 훈련된 모델과 토크나이저 불러오기
model_name = "noahkim/KoT5_news_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 사용자 입력으로부터 상품명 받기
title = input("상품명을 입력하세요: ")

# 해당 상품명에 해당하는 리뷰들을 모아서 리스트로 가져옴
reviews = df[df['상품명'] == title]['리뷰'].to_list()

# 리뷰를 문자열로 변환
reviews_text = ' '.join(reviews)

# 리뷰 텍스트를 토큰화하여 모델에 입력으로 전달하고 요약 생성
inputs = tokenizer(reviews_text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=10, max_length=30, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)